# Project 3: 'AskFeminists' vs. 'MensRights'
## Part C: Vectorizing & Analysis

[**Vectorizing**](#vector)

Before combining the two subreddit dataframes into one for modeling, I tested out different vectorizing options on both to compare the two subreddits. I tried out CountVectorizer and TFIDF on the AskFeminists content setting ngrams = (3-5) to get the top phrases from both: TFIDF was less repetitive than the CountVectorizer so I used TFIDF for vectorizing throughout. 

I tried vectorizing on ngrams 3-5, 1-2, and just single words, and created two custom lists of stopwords: one with all English stopwords and words in common between the top 100 words (without stopwords) from each subreddit; the second list of stopwords was created after fitting TFIDF on both subreddits (with stopwords = the first custom list) and taking the common words from the top 100 lists of words for each subreddit again. I tested models with both sets of stopwords and ultimately they didn't improve the model over using stopwords = 'english.')

[**Modeling**](#model)

I primarily tested two models on the lemmatized text: Logistic Regression and Random Forest Classifier. Starting with Logistic Regression, I tried several different parameters for vectorizing with TFIDF and the best parameters were 125,000 max features, ngrams = 1-4, and stopwords = 'English'.

The best test score I had ended up being on Logistic Regression (76% accuracy compared to baseline of 50%). This model was overfit on the training data (85% accuracy), and setting lower max_features closed the gap to a 3% difference between training/testing scores but also lowered the test scores. 

The best training score I had was with RandomForestClassifier, n_estimators = 100 and max_depth = 880, which gave me 98% on training data (but just 72% on the test data.)

In [230]:
# Import libaries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score

import regex as re
import nltk
from nltk.corpus import stopwords # Import the stop word list

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_extraction import text

from sklearn.linear_model import LogisticRegression

pd.set_option('display.max_colwidth', -1)
pd.options.display.max_columns = 999

In [2]:
men = pd.read_csv('./men_clean_lem')
fem = pd.read_csv('./fem_clean_lem')

### Balancing datasets, removing nulls

In [3]:
men['lems'].isnull().sum()

225

In [5]:
men.dropna(inplace = True)

In [4]:
fem['lems'].isnull().sum()

350

In [6]:
fem.dropna(inplace = True)

In [7]:
men.shape

(31221, 8)

In [8]:
fem.shape

(28955, 8)

In [10]:
# creating smaller version for mensrights, same # rows as submissions for askfeminists
men = men.sample(29000, replace = False, random_state=42)

In [11]:
men.shape

(29000, 8)

## Vectorizing: Separate Analysis <a name="vector"></a>

In [49]:
# Instantiate a CountVectorizer
vect = CountVectorizer(ngram_range=(3,5), max_features = 10000, stop_words = 'english')
# Instantiate TFIDF
tfidf = TfidfVectorizer(ngram_range = (3, 5), max_features = 10000, stop_words = 'english')

In [29]:
# vectorizing fem for test
fem_vect = vect.fit_transform(fem['lems'])
# tfidfing fem for test
fem_tfidf = tfidf.fit_transform(fem['lems'])

In [30]:
# creating a df for vectorized words
fem_vect_df = pd.DataFrame(fem_vect.toarray(), columns=vect.get_feature_names())
# creating a df for tfidf words
fem_tfidf_df = pd.DataFrame(fem_tfidf.toarray(), columns=tfidf.get_feature_names())

In [31]:
# looking at vectorized value counts
vect_counts = fem_vect_df.sum(axis=0)
vect_counts.sort_values(ascending=False)[0:10]

reflect feminist perspective                  160
feminist reflect feminist                     155
feminist reflect feminist perspective         153
come feminist reflect                         89 
come feminist reflect feminist                89 
come feminist reflect feminist perspective    89 
level comment thread                          81 
difference men woman                          80 
feminist perspective comment                  74 
doesn make sense                              74 
dtype: int64

In [32]:
# looking at tfidf value counts
tfidf_counts = fem_tfidf_df.sum(axis=0)
tfidf_counts.sort_values(ascending=False)[0:10]

doesn make sense                41.643637
false rape accusation           36.430295
difference men woman            32.162463
traditional gender role         28.743795
just don think                  27.087392
reflect feminist perspective    26.147715
don really know                 25.665504
innocent proven guilty          24.234714
rape sexual assault             24.215631
gt don think                    23.785815
dtype: float64

TFIDF seems to have done a better job of ignoring similar phrases than count vectorizer (ex. 'reflect feminist perspective') and the results are more interesting.

##### Comparing Top Phrases (TFIDF, ngrams 3-5)

In [50]:
# fitting to tfidf
men_tfidf = tfidf.fit_transform(men['lems'])

In [51]:
# creating a df for tfidf words
men_tfidf_df = pd.DataFrame(men_tfidf.toarray(), columns=tfidf.get_feature_names())

In [52]:
# pulling top 10 phrases for men TFIDF with English stopwords, 3 are the same as fem
tfidf_counts_men = men_tfidf_df.sum(axis=0)
tfidf_counts_men.sort_values(ascending=False)[0:10]

false rape accusation     62.306164
men right movement        60.595215
men right issue           54.943756
international men day     50.388507
men right activist        39.744089
gender pay gap            33.950820
innocent proven guilty    33.152104
pay child support         27.741176
year old boy              22.977497
doesn make sense          22.798229
dtype: float64

#### Analyzing single words and bigrams

In [64]:
tfidf = TfidfVectorizer(ngram_range = (1, 2), max_features = 10000, stop_words = 'english')

In [65]:
# fitting to tfidf
fem_tfidf = tfidf.fit_transform(fem['lems'])

In [66]:
men_tfidf = tfidf.fit_transform(men['lems'])

In [67]:
# creating a df for tfidf words
fem_tfidf_df = pd.DataFrame(fem_tfidf.toarray(), columns=tfidf.get_feature_names())
men_tfidf_df = pd.DataFrame(men_tfidf.toarray(), columns=tfidf.get_feature_names())

In [68]:
# pulling top 10 words/phrases for fem TFIDF with English stopwords
tfidf_counts_fem = fem_tfidf_df.sum(axis=0)
tfidf_counts_fem.sort_values(ascending=False)[0:10]

wife ha             976.118614
men family          729.171984
fellow              677.549175
thing               612.694714
divorce             607.185712
partner violence    605.775949
lifestyle           550.282282
june                537.683578
voting              513.169382
greatest            450.885487
dtype: float64

In [69]:
# pulling top 10 words/phrases for men TFIDF with English stopwords
# interesting that feminist is one of the top 10 words for mensrights
tfidf_counts_men = men_tfidf_df.sum(axis=0)
tfidf_counts_men.sort_values(ascending=False)[0:10]

woman       868.976790
men         826.650364
wa          579.965131
just        491.954841
like        476.984193
don         462.072782
people      423.281936
gt          407.147621
right       403.265087
feminist    391.284568
dtype: float64

##### Comparing single words & bigrams

In [70]:
# finding top ngrams 1-2
top_100_fem_ngrams_stop = tfidf_counts_fem.sort_values(ascending=False)[0:100]
top_100_men_ngrams_stop = tfidf_counts_men.sort_values(ascending=False)[0:100]

In [71]:
# finding similar ngrams in both
fem_men_ngrams = set(top_100_men_ngrams_stop.index) & set(top_100_fem_ngrams_stop.index)

In [72]:
# not a lot of similarity in smaller ngrams
fem_men_ngrams

{'thing'}

There isn't a lot of similarity when comparing ngrams 1-2 between the two subs - AskFeminists has more common 2-word phrases (with stopwords removed) which suggest more similar contextualization of common words such as 'men' and 'women'. MensRights has very few 2-word phrases that are common, so the top 100 words/short phrases come with less context than the list for AskFeminists.

#### Analyzing top words (ngrams=1)

In [121]:
tfidf1 = TfidfVectorizer(max_features = 50000, stop_words = 'english')

In [125]:
# fitting to tfidf
fem_tfidf = tfidf1.fit_transform(fem['lems'])

In [127]:
men_tfidf = tfidf1.fit_transform(men['lems'])

In [126]:
# creating a df for tfidf words
fem_tfidf_df = pd.DataFrame(fem_tfidf.toarray(), columns=tfidf1.get_feature_names())

In [128]:
men_tfidf_df = pd.DataFrame(men_tfidf.toarray(), columns=tfidf1.get_feature_names())

In [129]:
# top words for askfeminists
tfidf_counts_fem = fem_tfidf_df.sum(axis=0)
tfidf_counts_fem.sort_values(ascending=False)[0:10]

woman       1058.470342
men         802.978127 
feminist    751.930089 
don         664.663076 
think       655.181627 
people      645.120908 
like        581.666010 
just        572.214255 
wa          530.030073 
gt          473.983548 
dtype: float64

In [130]:
# top words for mensrights
tfidf_counts_men = men_tfidf_df.sum(axis=0)
tfidf_counts_men.sort_values(ascending=False)[0:10]

woman       929.135323
men         889.633840
wa          594.643131
just        509.152025
don         493.963959
like        489.428134
people      441.916170
right       433.626604
feminist    407.702110
gt          406.739804
dtype: float64

In [135]:
# finding top ngrams 1-2
top_100_fem_words_stop = tfidf_counts_fem.sort_values(ascending=False)[0:100]
top_100_men_words_stop = tfidf_counts_men.sort_values(ascending=False)[0:100]

In [136]:
# finding similar words in both
fem_men_words = set(top_100_men_words_stop.index) & set(top_100_fem_words_stop.index)

In [137]:
len(fem_men_words) # 77 of the words are in the top 100 for both! 

77

In [138]:
fem_men_words

{'actually',
 'agree',
 'bad',
 'believe',
 'better',
 'case',
 'child',
 'come',
 'comment',
 'did',
 'didn',
 'doe',
 'doesn',
 'don',
 'equality',
 'fact',
 'feel',
 'female',
 'feminism',
 'feminist',
 'gender',
 'girl',
 'going',
 'good',
 'group',
 'gt',
 'guy',
 'ha',
 'having',
 'help',
 'isn',
 'issue',
 'just',
 'know',
 'life',
 'like',
 'look',
 'lot',
 'make',
 'male',
 'man',
 'mean',
 'men',
 'need',
 'people',
 'person',
 'point',
 'post',
 'problem',
 'rape',
 'read',
 'really',
 'reason',
 'right',
 'said',
 'say',
 'saying',
 'sex',
 'sexual',
 'society',
 'sure',
 'thanks',
 'thing',
 'think',
 'thought',
 'time',
 'use',
 'victim',
 'wa',
 'want',
 'way',
 'white',
 'woman',
 'word',
 'work',
 'wrong',
 'yes'}

#### Comparing Stopwords with Count Vect

In [104]:
vect1 = CountVectorizer(max_features = 50000, stop_words = 'english')

In [105]:
# fitting to vect
fem_vect = vect1.fit_transform(fem['lems'])

In [107]:
men_vect = vect1.fit_transform(men['lems'])

In [106]:
# creating a df for tfidf words
fem_vect_df = pd.DataFrame(fem_vect.toarray(), columns=vect1.get_feature_names())

In [108]:
men_vect_df = pd.DataFrame(men_vect.toarray(), columns=vect1.get_feature_names())

In [109]:
# top words for askfeminists
vect_counts_fem = fem_vect_df.sum(axis=0)
vect_counts_fem.sort_values(ascending=False)[0:10]

woman       22124
men         14360
people      12063
don         11036
think       10884
like        10296
feminist    10242
just        9659 
wa          9085 
gt          7853 
dtype: int64

In [110]:
# top words for mensrights
vect_counts_men = men_vect_df.sum(axis=0)
vect_counts_men.sort_values(ascending=False)[0:10]

woman       17622
men         16400
wa          10868
just        7402 
like        6929 
people      6385 
don         6367 
gt          6236 
right       5382 
feminist    5265 
dtype: int64

In [111]:
# finding top words
top_100_fem_words_stop = vect_counts_fem.sort_values(ascending=False)[0:100]
top_100_men_words_stop = vect_counts_men.sort_values(ascending=False)[0:100]

In [112]:
# finding similar words in both
fem_men_words = set(top_100_men_words_stop.index) & set(top_100_fem_words_stop.index)

In [114]:
len(fem_men_words) # about the same as tfidf

78

#### Setting Custom Stopwords, Second Comparison

In [ ]:
# Brian's code, setting stop words equal to intersection terms in top 100 using count vectorizer
stop_words_1 = text.ENGLISH_STOP_WORDS.union(['actually',
 'agree',
 'bad',
 'believe',
 'better',
 'case',
 'child',
 'come',
 'comment',
 'did',
 'didn',
 'doe',
 'doesn',
 'don',
 'equality',
 'fact',
 'feel',
 'female',
 'feminism',
 'feminist',
 'gender',
 'girl',
 'going',
 'good',
 'group',
 'gt',
 'guy',
 'ha',
 'having',
 'help',
 'isn',
 'issue',
 'just',
 'know',
 'life',
 'like',
 'look',
 'lot',
 'make',
 'male',
 'man',
 'mean',
 'men',
 'need',
 'people',
 'person',
 'point',
 'post',
 'problem',
 'rape',
 'read',
 'really',
 'reason',
 'right',
 'said',
 'say',
 'saying',
 'sex',
 'sexual',
 'society',
 'sure',
 'thanks',
 'thing',
 'think',
 'thought',
 'time',
 'use',
 'victim',
 'wa',
 'want',
 'way',
 'white',
 'woman',
 'word',
 'work',
 'wrong',
 'yes'])

In [164]:
tfidf2 = TfidfVectorizer(max_features = 50000, stop_words = stop_words_1)

In [165]:
# fitting to tfidf
fem_tfidf = tfidf2.fit_transform(fem['lems'])

In [167]:
men_tfidf = tfidf2.fit_transform(men['lems'])

In [166]:
# creating a df for tfidf words
fem_tfidf_df = pd.DataFrame(fem_tfidf.toarray(), columns=tfidf2.get_feature_names())

In [168]:
men_tfidf_df = pd.DataFrame(men_tfidf.toarray(), columns=tfidf2.get_feature_names())

In [170]:
# top words for askfeminists, after removing most common words
tfidf_counts_fem = fem_tfidf_df.sum(axis=0)
tfidf_counts_fem.sort_values(ascending=False)[0:10]

question      377.321037
yes           252.605901
idea          223.008711
different     218.713998
agree         211.648976
understand    204.031619
answer        201.841974
opinion       195.025050
example       191.834915
trans         182.431184
dtype: float64

In [171]:
# top words for mensrights, after removing most common words
tfidf_counts_men = men_tfidf_df.sum(axis=0)
tfidf_counts_men.sort_values(ascending=False)[0:10]

day        203.099967
archive    202.091099
boy        192.386067
law        172.444206
sub        169.660586
got        163.834252
article    163.380445
shit       163.243384
care       159.536521
yes        157.502748
dtype: float64

In [173]:
# finding top words
top_100_fem_words_stop1 = tfidf_counts_fem.sort_values(ascending=False)[0:100]
top_100_men_words_stop1 = tfidf_counts_men.sort_values(ascending=False)[0:100]

In [174]:
# finding similar words in both
fem_men_words1 = set(top_100_men_words_stop1.index) & set(top_100_fem_words_stop1.index)

In [176]:
len(fem_men_words1)

62

In [177]:
fem_men_words1

{'agree',
 'aren',
 'argument',
 'article',
 'assault',
 'best',
 'boy',
 'care',
 'change',
 'claim',
 'consent',
 'crime',
 'day',
 'different',
 'doing',
 'equal',
 'evidence',
 'exactly',
 'example',
 'far',
 'friend',
 'getting',
 'got',
 'hard',
 'hate',
 'idea',
 'job',
 'kind',
 'law',
 'let',
 'making',
 'masculinity',
 'matter',
 'maybe',
 'movement',
 'opinion',
 'place',
 'power',
 'pretty',
 'probably',
 'question',
 'read',
 'real',
 'seen',
 'sound',
 'stop',
 'study',
 'sub',
 'talk',
 'talking',
 'tell',
 'thanks',
 'toxic',
 'true',
 'try',
 'understand',
 'used',
 'violence',
 'word',
 'wouldn',
 'yeah',
 'yes'}

### Setting Custom Stopwords (2nd layer)

In [178]:
# added common words from last version to common words from previous version
stop_words_2 = text.ENGLISH_STOP_WORDS.union(['agree',
 'aren',
 'argument',
 'article',
 'assault',
 'best',
 'boy',
 'care',
 'change',
 'claim',
 'consent',
 'crime',
 'day',
 'different',
 'doing',
 'equal',
 'evidence',
 'exactly',
 'example',
 'far',
 'friend',
 'getting',
 'got',
 'hard',
 'hate',
 'idea',
 'job',
 'kind',
 'law',
 'let',
 'making',
 'masculinity',
 'matter',
 'maybe',
 'movement',
 'opinion',
 'place',
 'power',
 'pretty',
 'probably',
 'question',
 'read',
 'real',
 'seen',
 'sound',
 'stop',
 'study',
 'sub',
 'talk',
 'talking',
 'tell',
 'thanks',
 'toxic',
 'true',
 'try',
 'understand',
 'used',
 'violence',
 'word',
 'wouldn',
 'yeah',
 'yes',
'actually',
 'agree',
 'bad',
 'believe',
 'better',
 'case',
 'child',
 'come',
 'comment',
 'did',
 'didn',
 'doe',
 'doesn',
 'don',
 'equality',
 'fact',
 'feel',
 'female',
 'feminism',
 'feminist',
 'gender',
 'girl',
 'going',
 'good',
 'gt',
 'guy',
 'ha',
 'isn',
 'issue',
 'just',
 'know',
 'life',
 'like',
 'look',
 'lot',
 'make',
 'male',
 'man',
 'mean',
 'men',
 'nan',
 'need',
 'people',
 'person',
 'point',
 'post',
 'problem',
 'rape',
 'read',
 'really',
 'reason',
 'right',
 'said',
 'say',
 'saying',
 'sex',
 'sexual',
 'sure',
 'thanks',
 'thing',
 'think',
 'time',
 'use',
 'wa',
 'want',
 'way',
 'white',
 'woman',
 'word',
 'work',
 'wrong',
 'yes', 'aren',
 'argument',
 'assault',
 'best',
 'boy',
 'care',
 'change',
 'claim',
 'consent',
 'day',
 'different',
 'doing',
 'equal',
 'evidence',
 'exactly',
 'far',
 'getting',
 'got',
 'group',
 'hate',
 'having',
 'help',
 'idea',
 'job',
 'kind',
 'law',
 'le',
 'let',
 'making',
 'matter',
 'maybe',
 'movement',
 'ok',
 'place',
 'power',
 'pretty',
 'probably',
 'question',
 'real',
 'society',
 'sound',
 'stop',
 'sub',
 'support',
 'talk',
 'talking',
 'tell',
 'thought',
 'true',
 'try',
 'trying',
 'understand',
 'used',
 'victim',
 'violence',
 'world',
 'wouldn',
 'yeah',
 'year'])

### Combining into one DF

In [179]:
femvmen = pd.concat([fem, men], axis=0, join='outer')

In [180]:
femvmen.shape

(57955, 8)

In [181]:
femvmen.columns

Index(['Unnamed: 0', 'text', 'type', 'subreddit', 'removed', 'deleted',
       'clean_text_stop', 'lems'],
      dtype='object')

In [182]:
femvmen = pd.get_dummies(femvmen, columns=['subreddit'], drop_first = True)

In [183]:
femvmen.drop(columns = "Unnamed: 0", inplace = True)

In [366]:
femvmen.reset_index(inplace = True)

In [381]:
femvmen = pd.DataFrame(femvmen)

In [368]:
femvmen.to_csv('./femvmen_lem')

In [369]:
femvmen.columns

Index(['index', 'text', 'type', 'removed', 'deleted', 'clean_text_stop',
       'lems', 'subreddit_MensRights'],
      dtype='object')

# Modeling <a name="model"></a>

#### Setting Vars/TrainTestSplit

In [518]:
X = femvmen['lems']
y = femvmen['subreddit_MensRights']

In [519]:
y.value_counts(normalize = True) # about 50% mensrights posts

1    0.500388
0    0.499612
Name: subreddit_MensRights, dtype: float64

In [520]:
#TRAIN TEST SPLIT
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, shuffle=True, 
                                                    stratify = y)

In [521]:
y_train.value_counts(normalize = True)

1    0.500388
0    0.499612
Name: subreddit_MensRights, dtype: float64

#### Vectorizing

In [273]:
# setting up multiple vectorizers with english stopwords, ngrams 1-4, different features
tfidf = TfidfVectorizer(ngram_range = (1, 4), max_features = 125000, stop_words = 
                        'english')
tfidf2 = TfidfVectorizer(ngram_range = (1, 4), max_features = 125000, stop_words = 
                        stop_words_1)
tfidf3 = TfidfVectorizer(ngram_range = (1, 4), max_features = 125000, stop_words = 
                        stop_words_2)
tfidf4 = TfidfVectorizer(ngram_range = (1, 4), max_features = 100000, stop_words = 
                        'english')
tfidf5 = TfidfVectorizer(ngram_range = (1, 4), max_features = 100000, stop_words = 
                        stop_words_1)
tfidf6 = TfidfVectorizer(ngram_range = (1, 4), max_features = 100000, stop_words = 
                        stop_words_2)
tfidf7 = TfidfVectorizer(ngram_range = (1, 4), max_features = 50000, stop_words = 
                        'english')
tfidf8 = TfidfVectorizer(ngram_range = (1, 4), max_features = 50000, stop_words = 
                        stop_words_1)
tfidf9 = TfidfVectorizer(ngram_range = (1, 4), max_features = 50000, stop_words = 
                        stop_words_2)
tfidf10 = TfidfVectorizer(ngram_range = (1, 4), max_features = 5000, stop_words = 
                        'english')
tfidf11 = TfidfVectorizer(ngram_range = (1, 4), max_features = 5000, stop_words = 
                        stop_words_1)
tfidf12 = TfidfVectorizer(ngram_range = (1, 4), max_features = 5000, stop_words = 
                        stop_words_2)

In [274]:
#fit transform train sets
X_train_tf = tfidf.fit_transform(X_train)
X_train_tf2 = tfidf2.fit_transform(X_train)
X_train_tf3 = tfidf3.fit_transform(X_train)
X_train_tf4 = tfidf4.fit_transform(X_train)
X_train_tf5 = tfidf5.fit_transform(X_train)
X_train_tf6 = tfidf6.fit_transform(X_train)
X_train_tf7 = tfidf7.fit_transform(X_train)
X_train_tf8 = tfidf8.fit_transform(X_train)
X_train_tf9 = tfidf9.fit_transform(X_train)
X_train_tf10 = tfidf10.fit_transform(X_train)
X_train_tf11 = tfidf11.fit_transform(X_train)
X_train_tf12 = tfidf12.fit_transform(X_train)

In [275]:
# transform test set
X_test_tf = tfidf.transform(X_test)
X_test_tf2 = tfidf2.transform(X_test)
X_test_tf3 = tfidf3.transform(X_test)
X_test_tf4 = tfidf4.transform(X_test)
X_test_tf5 = tfidf5.transform(X_test)
X_test_tf6 = tfidf6.transform(X_test)
X_test_tf7 = tfidf7.transform(X_test)
X_test_tf8 = tfidf8.transform(X_test)
X_test_tf9 = tfidf9.transform(X_test)
X_test_tf10 = tfidf10.transform(X_test)
X_test_tf11 = tfidf11.transform(X_test)
X_test_tf12 = tfidf12.transform(X_test)

### Logistic Regression

Best versions of vectorized data for logistic regression are with stopwords = 'english' and max features = 125000. With very few features (10K or less) the gap between train and test scores closes to 3% but the test score continues to drop with fewer than 100K features.

In [522]:
logreg = LogisticRegression(penalty='l2')

In [524]:
# version 1
X_train_tf = tfidf.fit_transform(X_train)

In [525]:
X_test_tf = tfidf.transform(X_test)

In [526]:
logreg.fit(X_train_tf, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [527]:
logreg.score(X_train_tf, y_train)

0.8545854542317315

In [528]:
logreg.score(X_test_tf, y_test)

0.7605038391855751

In [279]:
# version 2
logreg.fit(X_train_tf2, y_train)
logreg.score(X_train_tf2, y_train)

0.8391640065568113

In [280]:
logreg.score(X_test_tf2, y_test)

0.7380726425675093

In [281]:
# version 3
logreg.fit(X_train_tf3, y_train)
logreg.score(X_train_tf3, y_train)

0.8343326719006126

In [282]:
logreg.score(X_test_tf3, y_test)

0.7316883789146752

In [283]:
# version 4
logreg.fit(X_train_tf4, y_train)
logreg.score(X_train_tf4, y_train)

0.8502070571995514

In [284]:
logreg.score(X_test_tf4, y_test)

0.7510999913726167

In [285]:
# version 5
logreg.fit(X_train_tf5, y_train)
logreg.score(X_train_tf5, y_train)

0.8355620740229488

In [286]:
logreg.score(X_test_tf5, y_test)

0.738849107065827

In [287]:
# version 6
logreg.fit(X_train_tf6, y_train)
logreg.score(X_train_tf6, y_train)

0.8304287809507377

In [288]:
logreg.score(X_test_tf6, y_test)

0.731860926580968

In [289]:
# version 7
logreg.fit(X_train_tf7, y_train)
logreg.score(X_train_tf7, y_train)

0.8377404883098956

In [290]:
logreg.score(X_test_tf7, y_test)

0.7490294193771029

In [291]:
# version 8
logreg.fit(X_train_tf8, y_train)
logreg.score(X_train_tf8, y_train)

0.824346475713916

In [292]:
logreg.score(X_test_tf8, y_test)

0.7391079285652662

In [293]:
# version 9
logreg.fit(X_train_tf9, y_train)
logreg.score(X_train_tf9, y_train)

0.8187818134759728

In [294]:
logreg.score(X_test_tf9, y_test)

0.7308256405832111

In [295]:
# version 10
logreg.fit(X_train_tf10, y_train)
logreg.score(X_train_tf10, y_train)

0.7789664394789061

In [296]:
logreg.score(X_test_tf10, y_test)

0.7393667500647054

In [297]:
# version 11
logreg.fit(X_train_tf11, y_train)
logreg.score(X_train_tf11, y_train)

0.7708135622465706

In [298]:
logreg.score(X_test_tf11, y_test)

0.7247001984298163

In [299]:
# version 12
logreg.fit(X_train_tf12, y_train)
logreg.score(X_train_tf12, y_train)

0.7659606591320852

In [300]:
logreg.score(X_test_tf12, y_test)

0.7149512552842723

### Most Influential Features

In [373]:
# fitting on best model
logreg.fit(X_train_tf, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [529]:
coefficients = logreg.coef_

In [530]:
coef_df = pd.DataFrame(coefficients, columns = tfidf2.get_feature_names())

In [531]:
coef_df.head()

,aa,aa policy,aaaaand,aabg,aap,aauw,ab,aback,abandon,abandoned,abandoning,abandonment,abbey,abbreviated,abbreviation,abc,abc news,abctv,abctv scottmorrisonmp,abdicate,abdicate responsibility,abdication,abdomen,abdominal,abducted,abduction,aberrant,aberration,abhor,abhorrent,abide,abide rule,abiding,abigail,ability,ability ability,ability associated,ability associated chess,ability associated chess assumption,ability based,ability birth,ability career,ability change,ability choose,ability coder,ability commit,ability communicate,ability compete,ability consent,ability control,ability decision,ability earn,ability experience,ability influence,ability job,ability lead,ability obtain,ability pay,ability perceive,ability power,ability prevent,ability provide,ability reproduce,ability responsibility,ability speak,ability survive,ability understand,ability wanted,ability willingness,ability willingness negotiate,ability willingness negotiate salary,abit,abit bbc,abject,abject poverty,ablated,able,able abandon,able abandon parental,able abandon parental obligation,able able,able abortion,able accept,able accept trash,able accept trash quality,able access,able accrue,able accrue wealth,able accrue wealth unlike,able achieve,able acknowledge,able act,able address,able afford,able afford able,able afford able day,able afford private,able agree,able analyze,able analyze situation,able angry,able angry allowed,able angry allowed arent,able answer,able answer question,able appeal,able apply,able appreciate,able argument,able ask,able assault,able assist,able assist husband,able assist husband trade,able avoid,able aware,able away,able baby,able beat,able block,able bodied,able bodied ci,able bodied disease,able bodied disease free,able bodied neurotypical,able body,able break,able break agreement,able breastfeed,able breathe,able bring,able build,able buy,able buy house,able care,able catch,able catch damn,able catch damn diagnostic,able change,able change maybe,able change mind,able choice,able choose,able choose father,able claim,able clear,able close,able comfortable,able comfortable home,able comfortable home guarantee,able company,able compete,able compete ci,able compete sport,able compete sport line,able complain,able concede,able concede status,able concede status father,able connect,able consent,able consent tell,able consider,able continue,able contribute,able control,able convince,able convince reddit,able cope,able court,able court order,able create,able cum,able data,able date,able day,able day recover,able day recover clinic,able deal,able decide,able decision,able defend,able definitely,able destroy,able determine,able directly,able discern,able discriminate,able discus,able discussion,able distinguish,able dress,able empathize,able evidence,able experience,able explain,able explain biological,able explain biological basis,able express,able fight,able financial,able force,able forward,able fully,able gain,able game,able handle,able happens,able hold,able identify,able incorporate,able infer,able infer kind,able infer kind iat,able inform,able interpret,able job,able laid,able learn,able leave,able legally,able leverage,able lift,able lift pound,able live,able ma,able ma line,able maintain,able manipulate,able marry,able meet,able million,able navigate,able note,able note moderator,able offer,able opt,able participate,able past,able pay,able perform,able pick,able play,able power,able pregnant,able protect,able prove,able provide,able pull,able read,able recognize,able record,able reduce,able reduced,able reduced free,able reduced free home,able refute,able relate,able remain,able remove,able run,able sell,able separate,able separate conversation,able separate conversation fgm,able set,able sexist,able share,able shut,able sign,able similar,able single,able solve,able speak,able speak clearly,able speak clearly unambiguously,able spend,able spend hour,able spot,able stand,able stay,able stop,able straight,able strong,able stud

In [532]:
# words most positively correlated with content being from MensRights
# some of these make NO sense, might have been good to do more cleaning
coef_df.sum(axis=0).sort_values(ascending=False)[0:15]

member dominant                      5.570271
bread scientist react violently      4.393380
argument moving                      4.075883
message author                       3.676436
sexually submissive                  3.567055
brave serve                          3.275661
mobile atm                           3.235209
manifesting                          3.220126
self righteousness simply            2.914714
atlassian accepted majority coder    2.782258
beg scrap                            2.760610
slut slut                            2.736139
expendable                           2.723877
month son                            2.716348
bias court                           2.708516
dtype: float64

In [533]:
# words most positively correlated with content being from AskFeminists
# these make a little more sense than the MensRights ones...slightly.
coef_df.sum(axis=0).sort_values(ascending=True)[0:15]

pre determined                  -5.681340
study seminar                   -4.972181
talking federation autonomous   -4.799818
eff                             -3.908666
anti sjw                        -3.638601
desperate attempt               -3.596841
shitty yeah                     -3.415897
pick apart                      -3.373890
okay sexualize                  -3.134356
exposed message                 -3.017257
brainwashing hell drug          -2.897521
duo                             -2.883798
distinguish                     -2.873333
non binary particularly         -2.842388
appearance body                 -2.806616
dtype: float64

### Confusion Matrix

In [534]:
pred_log = logreg.predict(X_test_tf)

In [535]:
pred_log.shape

(11591,)

In [536]:
y_test.shape

(11591,)

In [537]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, pred_log)
tn, fp, fn, tp = confusion_matrix(y_test, pred_log).ravel()
print("True Negatives: %s" % tn)
print("False Positives: %s" % fp)
print("False Negatives: %s" % fn)
print("True Positives: %s" % tp)

True Negatives: 4317
False Positives: 1474
False Negatives: 1302
True Positives: 4498


In [538]:
# setting up df for comparison
y_test_preds = pd.DataFrame(y_test)

In [539]:
y_test_preds['pred'] = pred_log

In [540]:
y_test_preds.head()

,subreddit_MensRights,pred
57021,1,0
46763,1,1
26190,0,0
37992,1,1
11353,0,0


In [541]:
# rows where predictions don't match
y_test_preds[y_test_preds['subreddit_MensRights'] != y_test_preds['pred']].head()

,subreddit_MensRights,pred
57021,1,0
53228,1,0
18964,0,1
49924,1,0
41747,1,0


In [542]:
# example of a post misclassified as AskFeminists
#this comment is just responding to another user who posted something about JP (Jordan Peterson)
# but there's no real context that would help identify which thread this belongs to
femvmen.loc[57021]

index                   19820                                                                                                                                                                                                                                                   
text                    The first clip legitimately doesn't work, and none from the second are JP so I've still not seen proof of your origional claim that JP ever said that. If you send me a functioning clip then I'll change my opinion but it just says video unavailable.
type                    comment                                                                                                                                                                                                                                                 
removed                 0                                                                                                                                                            

In [543]:
# example of a post misclassified as MensRights
# not much to go off here for classification, it could really be from either.
femvmen.loc[18964]

index                   19170                                                                     
text                    Dude, I can't argue with you when you are so deep in the dogma. Good luck.
type                    comment                                                                   
removed                 0                                                                         
deleted                 0                                                                         
clean_text_stop         dude  i can t argue with you when you are so deep in the dogma  good luck 
lems                    dude i can argue with you when you are so deep in the dogma good luck     
subreddit_MensRights    0                                                                         
Name: 18964, dtype: object

### Other models

I tried all of the below models before selecting the Logistic Regression for further analysis of the coefficients - none of these had as strong a test score as the one on the Logistic Regression. I spent more time with Random Forest, and I could have tried to tune the parameters for Naive Bayes and SVC to improve the scores but didn't have time.

### Naive Bayes Multinomial

In [386]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()

In [387]:
nb.fit(X_train_tf2, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [388]:
nb.score(X_train_tf2, y_train)

0.8503796048658442

In [389]:
nb.score(X_test_tf2, y_test)

0.5014235182469157

## Support Vector Classifier

In [390]:
from sklearn import svm
svc = svm.SVC(kernel= 'rbf', C = 100, gamma = 0.05)

In [391]:
svc.fit(X_train_tf, y_train)

SVC(C=100, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.05, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [392]:
svc.score(X_train_tf, y_train)

0.9543180053489777

In [393]:
svc.score(X_test_tf, y_test)

0.4925373134328358

## Random Forest

In [394]:
from sklearn.ensemble import RandomForestClassifier

In [455]:
# taking sample for faster gridsearching
sample20000 = femvmen.sample(n=20000, random_state = 42)

In [456]:
sample20000['subreddit_MensRights'].value_counts(normalize=True) # about same proportions as full dataset

1    0.5018
0    0.4982
Name: subreddit_MensRights, dtype: float64

In [457]:
X = sample20000['lems']
y = sample20000['subreddit_MensRights']

In [458]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size = .30, 
                                                   stratify=y)

In [459]:
tfidf = TfidfVectorizer(ngram_range = (1, 4), max_features = 125000, stop_words = 
                        'english')

In [460]:
X_train_tf = tfidf.fit_transform(X_train)
X_test_tf = tfidf.transform(X_test)

In [461]:
X_train_tf.shape

(14000, 125000)

In [448]:
# just trying out basic random forest without tuning parameters or setting max depth
rf = RandomForestClassifier()

In [449]:
rf.fit(X_train_tf, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [450]:
rf.score(X_train_tf, y_train)

0.9857142857142858

In [451]:
rf.score(X_test_tf, y_test)

0.6108333333333333

In [415]:
rf = RandomForestClassifier()

In [416]:
# gridsearching for best parameters - this is the first one I did
from sklearn.model_selection import GridSearchCV

In [419]:
params = {
    'n_estimators': [5, 10, 1],
    'max_depth': [10000, 90000, 20000],
    'oob_score': ['True', 'False'],
    'warm_start': ['True'],
    'n_jobs': [-2]
}
gs = GridSearchCV(rf, param_grid = params, cv = 3)
gs.fit(X_train_tf, y_train)
print(gs.best_score_)
gs.best_params_ 

/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:453: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:458: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:453: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:458: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:453: UserWarning: Some inputs do not have OOB scores. This probably means too few trees we

/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:453: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:458: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:453: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:458: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:453: UserWarning: Some inputs do not have OOB scores. This probably means too few trees we

/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:453: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:458: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:453: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:458: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:453: UserWarning: Some inputs do not have OOB scores. This probably means too few trees we

0.6128571428571429


/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:453: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:458: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])


{'max_leaf_nodes': 20000,
 'n_estimators': 10,
 'n_jobs': -2,
 'oob_score': 'False',
 'warm_start': 'True'}

In [462]:
# second gridsearch, I repeated this until I narrowed down the best max depth range
params = {
    'n_estimators': [10],
    'max_depth': range(875, 900, 1),
    'oob_score': ['False'],
    'warm_start': ['True'],
    'n_jobs': [-2]
}
gs = GridSearchCV(rf, param_grid = params, cv = 3)
gs.fit(X_train_tf, y_train)
print(gs.best_score_)
gs.best_params_ 

/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:453: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:458: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:453: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:458: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:453: UserWarning: Some inputs do not have OOB scores. This probably means too few trees we

/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:453: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:458: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:453: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:458: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:453: UserWarning: Some inputs do not have OOB scores. This probably means too few trees we

/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:453: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:458: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:453: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:458: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:453: UserWarning: Some inputs do not have OOB scores. This probably means too few trees we

/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:453: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:458: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:453: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:458: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:453: UserWarning: Some inputs do not have OOB scores. This probably means too few trees we

0.6589285714285714


/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:453: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:458: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])


{'max_depth': 880,
 'n_estimators': 10,
 'n_jobs': -2,
 'oob_score': 'False',
 'warm_start': 'True'}

In [463]:
# resetting X and y variables to test random forest on full train/test sets
X = femvmen['lems']
y = femvmen['subreddit_MensRights']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, test_size = .25,
                                                   stratify = y)

In [464]:
X_train_tf = tfidf.fit_transform(X_train)
X_test_tf = tfidf.transform(X_test)

In [481]:
rf = RandomForestClassifier(max_depth = 880, n_estimators = 1000, n_jobs = -2, oob_score = False,
                 warm_start = True)

In [482]:
rf.fit(X_train_tf, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=880, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=-2,
            oob_score=False, random_state=None, verbose=0, warm_start=True)

In [483]:
rf.score(X_train_tf, y_train)

0.9846776791055077

In [484]:
rf.score(X_test_tf, y_test)

0.7199254606943198

## Conclusion

It would be too easy to say that my best model's inability to crack 77% accuracy means that "we're really not that different after all", and if I were to spend more time on this project there are other options I would explore to try and improve my model:

1. I'd pull a completely unrelated subreddit to test as a control against the other two subreddits to make sure I was tuning the best model possible.

2. Given how important context is for the comments, I would either try to aggregate comments on a post and analyze them together, or set a minimum wordcount on comments to be used in the analysis to give the model a better chance of distinguishing them.

3. I could scrape another years' worth of data to add to the model.

4. I could do some more cleaning and EDA that might help consolidate slang/similar terms that weren't captured by the lemmatizer, and identify stronger trends in the subreddits that could be leveraged for better prediction. It'd also be interesting to see what difference it might make to fit the TFIDF on just one subreddit first, then transform both subreddits and analyze them together.

5. If any of the above helped minimize overfit on the Logistic Regression or Random Forest, I'd give Naive Bayes Multinomial and Support Vector Classifier another shot (and spend more time tuning them).

The really interesting questions this project has generated would require a deeper analysis: what's the overlap in people who post on MensRights and AskFeminists? (How many of those are trolls, whose content is removed?) What common themes exist between removed posts (can we write an algorithm for trolling content?) How does sentiment analysis compare across the two subreddits, is there a discernable difference? Is it possible to measure 'extreme' attitudes in a subreddit, and if so can it be mapped over time, against real events happening in the world that might trigger anger on both or one side? Is it possible to follow the posts of single users and see how they change in sentiment and neutrality over time (and does the sub they post on the most make a difference?) Maybe I'll come back to it when I've solidified my modeling skills.